# Holder Wallet Conformance Flows (v3) - CTWalletCrossPreAuthorised

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

key_did = KeyDid()

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbncTAAd56KzjYx2Uvmt5diJYRjQW97m3Bja4m7QV4zcxSz5z7ChY5E9DtPCiqafTU9ksMGnN2BdU9nh62wY3MgtUYQWsqRqbZJL2U1BXRoXsRQzacrdsatqjskdwbGvGCsN


## 1.1 Initiate Credential Issuance

In [3]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletCrossPreAuthorised
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[18:39:46] Credential offer URI:                                                                    ]8;id=258038;file:///tmp/ipykernel_44505/724026412.py\724026412.py]8;;\:]8;id=971029;file:///tmp/ipykernel_44505/724026412.py#15\15]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/4b424fc6-3a55-4fb4-833                
           f-d0e9ac6ff6ef                                                                                          

[18:39:47] Credential offer:                                                                        ]8;id=704618;file:///tmp/ipykernel_44505/724026412.py\724026412.py]8;;\:]8;id=236727;file:///tmp/ipykernel_44505/724026412.py#18\18]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossPreAuthorised'                                                            
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'urn:ietf:params:oauth:grant-type:pre-authorized_code': {                                       
                       'pre-authorized_code':                                                                      
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZ                
           qcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1Bn                
           WW95dHlrVVozZXlxaHQxajlLYm5jVEFBZDU2S3pqWXgyVXZtdDVkaUpZUmpRVzk3bTNCamE0bTdRVjR6Y3hTejV6                
           N0NoWTVFOUR0UENpcWFmVFU5a3NNR25OMkJkVTluaDYyd1kzTWd0VVlRV3NxUnFiWkpMMlUxQlhSb1hzUlF6YWNy                
           ZHNhdHFqc2tkd2JHdkdDc04iLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVu                
           dGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNp                
           LmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwi                
           VmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJpYXQiOjE2ODcx                
           ODAxODYsImV4cCI6MTY4NzE4MDQ4NiwiaXNzIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25m                
           b3JtYW5jZS92My9pc3N1ZXItbW9jayIsImF1ZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29u                
           Zm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQ                
           Z1lveXR5a1VaM2V5cWh0MWo5S2JuY1RBQWQ1Nkt6all4MlV2bXQ1ZGlKWVJqUVc5N20zQmphNG03UVY0emN4U3o1                
           ejdDaFk1RTlEdFBDaXFhZlRVOWtzTUdu

           Credential issuer configuration:                                                         ]8;id=558969;file:///tmp/ipykernel_44505/724026412.py\724026412.py]8;;\:]8;id=129024;file:///tmp/ipykernel_44505/724026412.py#21\21]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cred                
           ential',                                                                                                
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-                
           mock/credential_deferred',                                                                              
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework': {

[18:39:48] Authorization server configuration:                                                      ]8;id=715063;file:///tmp/ipykernel_44505/724026412.py\724026412.py]8;;\:]8;id=772593;file:///tmp/ipykernel_44505/724026412.py#24\24]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/aut                
           horize',                                                                                                
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Exchange code for access token

In [4]:
from ebsi_wallet.siop_auth.util import exchange_pre_authorized_code_for_access_token

token_uri = auth_server_configuration.token_endpoint
user_pin = "1221"
access_token_response = await exchange_pre_authorized_code_for_access_token(token_uri, user_pin, credential_offer.grants.get("urn:ietf:params:oauth:grant-type:pre-authorized_code").get("pre-authorized_code"))
console.log("Access token response: ", access_token_response)

{'error': 'invalid_request', 'error_description': 'Invalid PIN'}


TypeError: AccessTokenResponse.__init__() got an unexpected keyword argument 'error'

## 1.3 Request credential (cross device)

In [ ]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
credential_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Credential response: ", credential_response)

[18:27:21] Credential request JWT:                                                                  ]8;id=995858;file:///tmp/ipykernel_42362/2423221428.py\2423221428.py]8;;\:]8;id=105107;file:///tmp/ipykernel_42362/2423221428.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtV                
           WjNleXFodDFqOUticmhCYUJmNlpQQXVYTXVLbVFDanprU1llVkxVNFZRYmI5R0hrUFVWTnpwZ3BhbXNUYlJOVXdS                
           YzU0Z3daSkFWcEFqakdIWWRFeVVRVGh3REN6ZHQ1Zm9CQTlGSHJqMzZOc0w2WlNVVURTTjR3ZTFrOWdCOFlTdG1O                
           Ukx3R25DQnIzNiN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JyaEJhQmY2                
           WlBBdVhNdUttUUNqemtTWWVWTFU0VlFiYjlHSGtQVVZOenBncGFtc1RiUk5Vd1JjNTRnd1pKQVZwQWpqR0hZZEV5                
           VVFUaHdEQ3pkdDVmb0JBOUZIcmozNk5zTDZaU1VVRFNONHdlMWs5Z0I4WVN0bU5STHdHbkNCcjM2IiwidHlwIjoi                
           b3BlbmlkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2Nvb                
           mZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3MjY1ODQxLCJpYXQiOjE2ODcxNzk0NDEsImlzcyI6I                
           mRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticmhCYUJmNlpQQ                
           XVYTXVLbVFDanprU1llVkxVNFZRYmI5R0hrUFVWTnpwZ3BhbXNUYlJOVXdSYzU0Z3daSkFWcEFqakdIWWRFeVVRV                
           Gh3REN6ZHQ1Zm9CQTlGSHJqMzZOc0w2WlNVVURTTjR3ZTFrOWdCOFlTdG1OUkx3R25DQnIzNiIsIm5vbmNlIjoiY                
           zA1OTMyOTAtN2VkMi00YTBlLWI4YzAtYmRhZWRjZDJlMDE4In0.CWLxQ_wzUq9eBZdVzpVciTd069fUlyyHpNTpP                
           C_YPM41-VXfgE1MnGwH5qkBVceSqMmTMo6-rW1ZeTrJnnCYBA                                                       

[18:27:22] Credential response:                                                                     ]8;id=779584;file:///tmp/ipykernel_42362/2423221428.py\2423221428.py]8;;\:]8;id=303603;file:///tmp/ipykernel_42362/2423221428.py#6\6]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWl                
           kI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNv                
           bmZvcm1hbmNlI2JlNTBlMmNlLThhNTctNDcyNC04ZDBiLWNlZDYxYTk5Y2RjNyIsInN1YiI6ImRpZDprZXk6ejJk                
           bXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticmhCYUJmNlpQQXVYTXVLbVFDanpr                
           U1llVkxVNFZRYmI5R0hrUFVWTnpwZ3BhbXNUYlJOVXdSYzU0Z3daSkFWcEFqakdIWWRFeVVRVGh3REN6ZHQ1Zm9C                
           QTlGSHJqMzZOc0w2WlNVVURTTjR3ZTFrOWdCOFlTdG1OUkx3R25DQnIzNiIsImlzcyI6ImRpZDplYnNpOnpoSkFS                
           alBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MTc5NDQyLCJleHAiOjE2ODcyNjU4NDIsImlhdCI6MTY4NzE3                
           OTQ0MiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwi                
           aWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2JlNTBlMmNlLThhNTctNDcyNC04ZDBiLWNlZDYxYTk5Y2RjNyIsInR5                
           cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3Nz                
           UHJlQXV0aG9yaXNlZCJdLCJpc3N1ZXIiOiJkaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V0Z1B4SGVuMU1pZCIsImlzc3Vh                
           bmNlRGF0ZSI6IjIwMjMtMDYtMTlUMTI6NTc6MjJaIiwiaXNzdWVkIjoiMjAyMy0wNi0xOVQxMjo1NzoyMloiLCJ2                
           YWxpZEZyb20iOiIyMDIzLTA2LTE5VDEyOjU3OjIyWiIsImV4cGlyYXRpb25EYXRlIjoiMjAyMy0wNi0yMFQxMjo1                
           NzoyMloiLCJjcmVkZW50aWFsU3ViamVjdCI6eyJpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1G                
           WXJXUGdZb3l0eWtVWjNleXFodDFqOUticmhCYUJmNlpQQXVYTXVLbVFDanprU1llVkxVNFZRYmI5R0hrUFVWTnpw                
           Z3BhbXNUYlJOVXdSYzU0Z3daSkFWcEFqakdIWWRFeVVRVGh3REN6ZHQ1Zm9CQTlGSHJqMzZOc0w2WlNVVURTTjR3                
           ZTFrOWdCOFlTdG1OUkx3R25DQnIzNiJ9LCJjcmVkZW50aWFsU2NoZW1hIjp7ImlkIjoiaHR0cHM6Ly9hcGktY29u                
           Zm9ybWFuY2UuZWJzaS5ldS90cnVzdGVkLXNjaGVtYXMtcmVnaXN0cnkvdjIvc2NoZW1hcy96M01nVUZVa2I3MjJ1                
           cTR4M2R2NXlBSm1uTm16REZlSzVVQzh4ODNRb2VMSk0iLCJ0eXBlIjoiRnVsbEpzb25TY2hlbWFWYWxpZGF0b3Iy                
           MDIxIn0sInRlcm1zT2ZVc2UiOnsiaWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L3RydXN0ZWQt                
           aXNzdWVycy1yZWdpc3RyeS92NC9pc3N1ZXJzL2RpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkL2F0dHJp                
           YnV0ZXMvZjFhY2IzNzVmY2ZiZTExYTYwMDNkNWI3MTBiZjM3ZWNiMjc4MmMxMWE0ZTQyODU3MDEzMzc3MzJjMjY2                
           NGQwMCIsInR5cGUiOiJJc3N1YW5jZUNlcnRpZmljYXRlIn19fQ.HKo-kyJIFqSDlpSshDF_5vOYWTFiYvy-zklJN                
           7WYs4AkwvIjFa7ong9JRQYF18tu-JhxuVswdID7GEpksvDPBA'                                                      
           }                                                                                                       